Character issues, Unicode

In [1]:
s = 'cafe'
len(s)

4

In [2]:
b = s.encode('utf-8')
b

b'cafe'

In [3]:
len(b)

4

In [4]:
b.decode('utf-8')

'cafe'

In [6]:
cafe = bytes('cafe', encoding='utf-8')
cafe

b'cafe'

In [8]:
cafe[0]

99

In [9]:
cafe[:1]

b'c'